In [38]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [39]:
citylist = ['Munich', 'Rotterdam-Hague', 'Frankfurt', 'Hamburg', 'Zurich']
UrbanTypo = pd.read_excel('Data/FINAL-COMBINED-DATASET.xlsx')
Cities = UrbanTypo.loc[(UrbanTypo.City.isin(citylist))]
Cities.reset_index(inplace = True)
Cities.head()

,index,City,cityID,clusterID,Typology,Country,Car Modeshare (%),Public Transit Modeshare (%),Bicycle Modeshare (%),Walking Modeshare (%),...,Highway Proportion,Metro Propensity Factor,BRT Propensity Factor,BikeShare Propensity Factor,Development Factor,Sustainability Factor,Population Factor,Congestion Factor,Sprawl Factor,Network Density Factor
0,98,Hamburg,99,9,MassTransit Heavyweight,Germany,42.00,18.00,12,28.00,...,0.016812,0.667477,0.273596,0.437481,0.747204,0.536597,0.035424,0.357805,0.313441,0.231415
1,99,Munich,100,9,MassTransit Heavyweight,Germany,37.00,21.00,14,28.00,...,0.021030,0.674947,0.142531,0.433690,0.779353,0.599366,0.033533,0.354032,0.236366,0.429755
2,101,Frankfurt,102,10,MassTransit Moderate,Germany,38.00,20.00,11,31.00,...,0.033763,0.080828,0.131759,0.403807,0.741574,0.625617,0.044672,0.323014,0.323657,0.342305
3,182,Rotterdam-Hague,183,10,MassTransit Moderate,Netherlands,49.00,17.00,16,18.00,...,NaN,0.356458,0.164029,0.484939,0.750362,0.540186,0.035450,0.204339,0.193488,0.508605
4,238,Zurich,239,10,MassTransit Moderate,Switzerland,42.92,46.31,4.14,5.65,...,0.025206,0.090735,0.346604,0.163072,1.000000,0.575431,0.025346,0.388853,0.185876,0.453369


In [40]:
for i in Cities.columns:
    print(i)

index
City
cityID
clusterID
Typology
Country
Car Modeshare (%)
Public Transit Modeshare (%)
Bicycle Modeshare (%)
Walking Modeshare (%)
Gasoline Pump Price (USD/liter)
Road Deaths Rate (per 1000)
Subway Length (km)
Subway Length Density (per km)
Subway Stations per Hundred Thousand
Subway Ridership per Capita
Subway Age (years)
BRT Length (km)
BRT System Length Density (per km)
BRT Stations per Hundred Thousand Persons
BRT Fleet per Hundred Thousand Persons
BRT Annual Ridership per Capita
BRT Age (years)
Bikeshare Stations
Bikeshare Stations per Hundred Thousand Persons
Bikeshare Number of Bikes
Bikeshare Bicycles per Hundred Thousand Persons
Bikeshare Age (years)
Congestion (%)
Congestion AM Peak (%)
Congestion PM Peak (%)
Traffic Index
Travel Time Index
Inefficiency Index
Population
Land Area (sq. km)
Population Density (per sq. km)
Population Change 1990 – 2000
Population Change 2000 – 2010
Population Change 2010 – 2020
Population Change 2020 – 2025
Urbanization Rate 2015 (%)
Urbani

After inspecting columns, sustainability appears to be expressed by the factors on a logical and comparable scale. Those are therefore chosen to be aggegrated. Let's subset those columns.

In [41]:
Cities_for_pca=Cities.iloc[:,70:79]
Cities_for_pca.head()

,Metro Propensity Factor,BRT Propensity Factor,BikeShare Propensity Factor,Development Factor,Sustainability Factor,Population Factor,Congestion Factor,Sprawl Factor,Network Density Factor
0,0.667477,0.273596,0.437481,0.747204,0.536597,0.035424,0.357805,0.313441,0.231415
1,0.674947,0.142531,0.433690,0.779353,0.599366,0.033533,0.354032,0.236366,0.429755
2,0.080828,0.131759,0.403807,0.741574,0.625617,0.044672,0.323014,0.323657,0.342305
3,0.356458,0.164029,0.484939,0.750362,0.540186,0.035450,0.204339,0.193488,0.508605
4,0.090735,0.346604,0.163072,1.000000,0.575431,0.025346,0.388853,0.185876,0.453369


First we have to std the values https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

In [42]:
#standardize values
Cities_for_pca_std=Cities_for_pca
for i in range(len(Cities_for_pca.columns)):
    Cities_for_pcastd.iloc[:,i] = (Cities_for_pca.iloc[:,i] - Cities_for_pca.iloc[:,i].mean()) / Cities_for_pca.iloc[:,i].std()


In [43]:
Cities_for_pcastd.head()

,City,Metro Propensity Factor,BRT Propensity Factor,BikeShare Propensity Factor,Development Factor,Sustainability Factor,Population Factor,Congestion Factor,Sprawl Factor,Network Density Factor
0,1.001618,0.657188,0.415754,-0.510315,-1.016389,0.078338,0.449105,0.966176,-1.490875,0.231415
1,1.027120,-0.734492,0.385948,-0.219915,0.626088,-0.196569,0.396479,-0.218204,0.338108,0.429755
2,-1.001184,-0.848875,0.151019,-0.561167,1.313008,1.422968,-0.036188,1.123158,-0.468309,0.342305
3,-0.060191,-0.506223,0.788856,-0.481788,-0.922485,0.082212,-1.691601,-0.877079,1.065213,0.508605
4,-0.967364,1.432402,-1.741578,1.773186,-0.000222,-1.386949,0.882205,-0.994052,0.555863,0.453369


In [44]:
pca = PCA(n_components=1) # we want 1 component
principalComponents = pca.fit_transform(Cities_for_pcastd)#lets fit

principalDf1 = pd.DataFrame(data = principalComponents
             , columns = ['Standardized aggegrated value'])
principalDf1.head()

,Standardized aggegrated value
0,-0.824332
1,-0.485794
2,-1.650407
3,-0.633201
4,3.593734


In [45]:
#What if we dont standardize?
pca = PCA(n_components=1) # we want 1 component
principalComponents = pca.fit_transform(Cities_for_pca)#lets fit

principalDf2 = pd.DataFrame(data = principalComponents
             , columns = ['Non standardized aggegrated value'])
principalDf2.head()

,Non standardized aggegrated value
0,-0.315891
1,-0.293719
2,0.232356
3,-0.015249
4,0.392503
